In [1]:
# coding: utf-8
import gym
import torch.utils.data as data
from dynamics import *
from controller import *
from utils import *
from quanser_robots.common import GentlyTerminating
import time

# datasets:  numpy array, size:[sample number, input dimension]
# labels:  numpy array, size:[sample number, output dimension]

env_id ="Qube-v0" # "CartPole-v0"
env = GentlyTerminating(gym.make(env_id))
config_path = "config.yml"
config = load_config(config_path)
#print_config(config_path)

In [2]:
model = DynamicModel(config)

data_fac = DatasetFactory(env,config)
data_fac.collect_random_dataset()

/home/xinye/projects/rl/RL-project/MPC/MPC qube/dynamics.py:209: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if datasets == None:
/home/xinye/projects/rl/RL-project/MPC/MPC qube/dynamics.py:213: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels == None:


Collect random dataset shape:  (7630, 7)


In [3]:
loss = model.train(data_fac.random_trainset,data_fac.random_testset)

Total training step per epoch [96]
Epoch [50/500], Training Loss: 0.07970656, Test Loss: 0.08558914
Epoch [100/500], Training Loss: 0.07005977, Test Loss: 0.08037411
Epoch [150/500], Training Loss: 0.06543648, Test Loss: 0.07438301
Epoch [200/500], Training Loss: 0.06311109, Test Loss: 0.07371941
Epoch [250/500], Training Loss: 0.06101681, Test Loss: 0.07367975
Epoch [300/500], Training Loss: 0.06066504, Test Loss: 0.07022873
Epoch [350/500], Training Loss: 0.05915687, Test Loss: 0.07054408
Epoch [400/500], Training Loss: 0.05823344, Test Loss: 0.06820911
Epoch [450/500], Training Loss: 0.05771950, Test Loss: 0.06866388
Epoch [500/500], Training Loss: 0.05728819, Test Loss: 0.06751502


In [ ]:
config["mpc_config"]["horizon"] = 12
config["mpc_config"]["numb_bees"] = 8
config["mpc_config"]["max_itrs"] = 20
config["mpc_config"]["gamma"] = 0.98
mpc = MPC(env,config)

In [ ]:
rewards_list = []
for itr in range(config["dataset_config"]["n_mpc_itrs"]):
    t = time.time()
    print("**********************************************")
    print("The reinforce process [%s], collecting data ..." % itr)
    rewards = data_fac.collect_mpc_dataset(mpc,model)
    trainset, testset = data_fac.make_dataset()
    rewards_list += rewards
    
    plt.close("all")
    plt.figure(figsize=(12, 5))
    plt.title('Reward Trend with %s iteration' % itr)
    plt.plot(rewards_list)
    plt.savefig("storage/reward-" + str(model.exp_number) + ".png")
    print("Consume %s s in this iteration" % (time.time()-t))
    loss = model.train(trainset,testset)

**********************************************
The reinforce process [0], collecting data ...


/home/xinye/projects/rl/RL-project/MPC/MPC qube/dynamics.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda()


Episode [0/6], Reward: 1.07203271
